In [48]:
import requests
import json

def get_transaction_details(intent_hash):
    """
    Fetch transaction details from the Radix network.
    
    Args:
        intent_hash (str): The transaction intent hash to query
        
    Returns:
        requests.Response: The response object from the API request
    """
    # Define the endpoint URL
    url = 'https://mainnet.radixdlt.com/transaction/committed-details'

    # Define the headers
    headers = {
        'accept': 'application/json',
        'Content-Type': 'application/json'
    }

    # Define the payload data
    payload = {
        "intent_hash": intent_hash,
        "opt_ins": {
            "raw_hex": True,
            "receipt_state_changes": True,
            "receipt_fee_summary": True,
            "manifest_instructions": True,
            "receipt_fee_source": True,
            "receipt_fee_destination": True,
            "receipt_costing_parameters": True,
            "receipt_events": True,
            "receipt_output": True,
            "affected_global_entities": True,
            "balance_changes": True
        }
    }

    # Make the POST request
    return requests.post(url, headers=headers, data=json.dumps(payload))


# Example usage
# response = get_transaction_details("txid_rdx1aqqwq3e95k4ktr5yt2s7sgtnsnf64uh87gase8k74we3fe5fyx3q7tzvls")
response = get_transaction_details("txid_rdx1arc8vukn4j5lg0vfrjdjxl3rf0e7uh8vc22t4wjhqq3s0pz6xnvqq6f88p")
# response = get_transaction_details("txid_rdx1nvh6xsj03ykm0r03ppny4qj9xtjtl9vaf4hw5n77ammg0sedr4zqnz43xf")
# response = get_transaction_details("txid_rdx16jethsv50jvj7nalsqqa2zwvzhps04avx9k5gal3qukgp3a8sjsqjq077a") # Adam's example txn

# Print the response status code and content
print(f"Status Code: {response.status_code}")
print(f"Response Content: {response.json()}")



Status Code: 200
Response Content: {'ledger_state': {'network': 'mainnet', 'state_version': 286255281, 'proposer_round_timestamp': '2025-05-13T09:42:59.889Z', 'epoch': 202957, 'round': 1406}, 'transaction': {'transaction_status': 'CommittedSuccess', 'state_version': 286232755, 'epoch': 202945, 'round': 185, 'round_timestamp': '2025-05-13T08:39:42.802Z', 'payload_hash': 'notarizedtransaction_rdx1083vywn2ch8du8agx5u3dcqk8zm3q56wtsa7eqp0rerxrhlr2neqx0cvep', 'intent_hash': 'txid_rdx1arc8vukn4j5lg0vfrjdjxl3rf0e7uh8vc22t4wjhqq3s0pz6xnvqq6f88p', 'fee_paid': '4.391931913313333333', 'affected_global_entities': ['transactiontracker_rdx1stxxxxxxxxxxtxtrakxxxxxxxxx006844685494xxxxxxxxxtxtrak', 'component_rdx1czre754pv64wd74v4mn80769jy5l3p63ecv9unzja7hemcmf02vvjh', 'consensusmanager_rdx1scxxxxxxxxxxcnsmgrxxxxxxxxx000999665565xxxxxxxxxcnsmgr', 'resource_rdx1t5pyvlaas0ljxy0wytm5gvyamyv896m69njqdmm2stukr3xexc2up9', 'component_rdx1czgaazn4wqf40kav57t8tu6kwv2a5sfmnlzlar9ee6kdqk0ll2chsz', 'component_rdx1

In [47]:
import pandas as pd

txndata = response.json()
events = txndata['transaction']['receipt']['events']
# Extract data from events
event_data = []

for event in events:
    # Initialize a dictionary for this event
    event_dict = {
        'name': event['name'],
        'emitter_type': event['emitter']['type'],
        'entity_type': event['emitter']['entity']['entity_type'],
        'entity_address': event['emitter']['entity']['entity_address'],
        'data_kind': event['data']['kind']
    }
    
    # Extract fields from data based on structure
    if event['data']['kind'] == 'Tuple' and 'fields' in event['data']:
        for field in event['data']['fields']:
            if 'field_name' in field:
                if 'value' in field:
                    event_dict[field['field_name']] = field['value']
    elif event['data']['kind'] == 'Enum' and 'fields' in event['data']:
        event_dict['variant_name'] = event['data'].get('variant_name', '')
        for field in event['data']['fields']:
            if isinstance(field, dict) and 'kind' in field:
                if 'field_name' in field:
                    event_dict[field['field_name']] = field['value']
                elif 'type_name' in field:
                    event_dict[field['type_name']] = field['value']
    
    event_data.append(event_dict)


# Create a pandas DataFrame
events_df = pd.DataFrame(event_data)

# Display the DataFrame
events_df

,name,emitter_type,entity_type,entity_address,data_kind,amount,variant_name,ResourceAddress,new_balance,resource_address,...,global_x_fee_lp,global_y_fee_lp,token_address,amount_change_x,amount_change_y,price_after,amount_after_x,amount_after_y,base_amount,quote_amount
0,LockFeeEvent,Method,InternalFungibleVault,internal_vault_rdx1tq6emj939c4qejwtd24jvfvhwq7...,Tuple,10,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,WithdrawEvent,Method,InternalFungibleVault,internal_vault_rdx1tz5mypwz7vzv3cxqwann0zkawey...,Tuple,0.02219,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,WithdrawEvent,Method,GlobalVirtualSecp256k1Account,account_rdx16ykn77yw3mnmjtc2q3dx5qnxml7gap33fp...,Enum,NaN,Fungible,resource_rdx1t5pyvlaas0ljxy0wytm5gvyamyv896m69...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,BurnFungibleResourceEvent,Method,GlobalFungibleResource,resource_rdx1t5pyvlaas0ljxy0wytm5gvyamyv896m69...,Tuple,0.02219,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,BurnEvent,Method,GlobalGenericComponent,component_rdx1crmhkatyjrw0070nsusdm4adwr5s3eay...,Tuple,0.02219,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,DepositEvent,Method,InternalFungibleVault,internal_vault_rdx1tzux9934ev23sy99plx5fv0e5r6...,Tuple,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,ReserveDepositEvent,Method,GlobalGenericComponent,component_rdx1crmhkatyjrw0070nsusdm4adwr5s3eay...,Tuple,0,NaN,NaN,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,WithdrawEvent,Method,InternalFungibleVault,internal_vault_rdx1tqt3q4aj92rgrqguvt3jahlpwuz...,Tuple,48630.228887498579994582,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,SwapEvent,Method,GlobalGenericComponent,component_rdx1crmhkatyjrw0070nsusdm4adwr5s3eay...,Tuple,48630.228887498579994582,NaN,NaN,NaN,resource_rdx1t466mhd2l2jmmzxr8cg3mkwjqhs7zmjgt...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,DepositEvent,Method,InternalFungibleVault,internal_vault_rdx1trs0xmx638vcn72e8uyc0ct020t...,Tuple,48.630228887498579994,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:

# Filter out SwapEvent from the dataframe
swap_events = events_df[events_df['name'].isin(['SwapEvent', 'SwapVaultDepositEvent'])]

# Select and display the requested columns
if not swap_events.empty:
    swap_details = swap_events[[
        'entity_address',
        'amount',
        'input_address',
        'input_amount',
        'input_fee_lp',
        'input_fee_protocol',
        'output_address',
        'output_amount'
    ]].reset_index(drop=True)
    
    print("Swap Event Details:")
    display(swap_details)
else:
    print("No SwapEvent found in the data.")


Swap Event Details:


,entity_address,amount,input_address,input_amount,input_fee_lp,input_fee_protocol,output_address,output_amount
0,component_rdx1cpgmgrskahkxe4lnpp9s2f5ga0z8jkl7...,NaN,resource_rdx1tknxxxxxxxxxradxrdxxxxxxxxx009923...,499.75,0.225,0.025,resource_rdx1t580qxc7upat7lww4l2c4jckacafjeudx...,0.00004459
